In [2]:
import csv
import time
import traceback
import pandas as pd
from googleapiclient.discovery import build

In [ ]:
API_KEY = 'api_code'
MAX_COMMENTS = 15000

youtube = build('youtube', 'v3', developerKey=API_KEY)

video_sources = {
    'gP0YCMCJnjA': 'Narasi Newsroom',
    '7lGiipi4czw': 'Kompas',
    'jiP96n4z4Lk': 'CNN Indonesia',
    '-WLpXmnBmxo': 'Najwa Shihab',
}

comments = []

In [4]:
def get_comments(video_id, source_name):
    next_page_token = None
    last_report = 0

    while len(comments) < MAX_COMMENTS:
        try:
            request = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                textFormat='plainText',
                maxResults=100,
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                top_comment_snippet = item['snippet']['topLevelComment']['snippet']
                author = top_comment_snippet.get('authorDisplayName', 'Anonim')
                comment_text = top_comment_snippet.get('textDisplay', '')
                comments.append((author, comment_text, source_name))

                # Ambil balasan komentar jika ada
                if item['snippet']['totalReplyCount'] > 0:
                    parent_id = item['snippet']['topLevelComment']['id']
                    reply_request = youtube.comments().list(
                        part='snippet',
                        parentId=parent_id,
                        textFormat='plainText',
                        maxResults=100
                    )
                    reply_response = reply_request.execute()

                    for reply in reply_response['items']:
                        reply_snippet = reply['snippet']
                        reply_author = reply_snippet.get('authorDisplayName', 'Anonim')
                        reply_text = reply_snippet.get('textDisplay', '')
                        comments.append((reply_author, reply_text, source_name))

                # Tampilkan progres setiap 1000 komentar
                if len(comments) // 1000 > last_report:
                    last_report = len(comments) // 1000
                    print(f"[INFO] Telah mengumpulkan {len(comments)} komentar...")

                if len(comments) >= MAX_COMMENTS:
                    break

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break

            time.sleep(0.1)

        except Exception as e:
            print(f"[ERROR] Terjadi kesalahan: {e}")
            traceback.print_exc()
            print("Menunggu 5 detik sebelum mencoba lagi...")
            time.sleep(5)

In [ ]:
print("[START] Mengambil komentar dari YouTube...")
for vid, source in video_sources.items():
    print(f"[INFO] Mengambil komentar dari sumber: {source} (videoId: {vid})")
    get_comments(vid, source)
    if len(comments) >= MAX_COMMENTS:
        break

with open('prabowo_menjawab.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Nama Akun', 'Komentar', 'Sumber'])
    for author, comment, source in comments:
        writer.writerow([author, comment, source])

print(f"[DONE] {len(comments)} komentar berhasil disimpan ke 'prabowo_menjawab.csv'")

[START] Mengambil komentar dari YouTube...
[INFO] Mengambil komentar dari sumber: Narasi Newsroom (videoId: gP0YCMCJnjA)
[INFO] Telah mengumpulkan 1017 komentar...
[INFO] Telah mengumpulkan 2000 komentar...
[INFO] Telah mengumpulkan 3000 komentar...
[INFO] Telah mengumpulkan 4014 komentar...
[INFO] Telah mengumpulkan 5000 komentar...
[INFO] Mengambil komentar dari sumber: Kompas (videoId: 7lGiipi4czw)
[INFO] Telah mengumpulkan 5758 komentar...
[INFO] Telah mengumpulkan 6000 komentar...
[INFO] Telah mengumpulkan 7001 komentar...
[INFO] Telah mengumpulkan 8003 komentar...
[INFO] Mengambil komentar dari sumber: CNN Indonesia (videoId: jiP96n4z4Lk)
[INFO] Telah mengumpulkan 8008 komentar...
[INFO] Telah mengumpulkan 9000 komentar...
[INFO] Mengambil komentar dari sumber: Najwa Shihab (videoId: -WLpXmnBmxo)
[INFO] Telah mengumpulkan 9012 komentar...
[INFO] Telah mengumpulkan 10000 komentar...
[INFO] Telah mengumpulkan 11000 komentar...
[INFO] Telah mengumpulkan 12000 komentar...
[INFO] Tela

In [7]:
data = pd.read_csv('prabowo_menjawab.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15020 entries, 0 to 15019
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nama Akun  15017 non-null  object
 1   Komentar   15016 non-null  object
 2   Sumber     15020 non-null  object
dtypes: object(3)
memory usage: 352.2+ KB


In [8]:
data.head(10)

,Nama Akun,Komentar,Sumber
0,@azzahrasyamilarahmat5028,11/100 Is Real...,Narasi Newsroom
1,@faritrohman3969,Saya kecewa cuma gara gara beberapa anak kelua...,Narasi Newsroom
2,@moradepadli5676,Prabowo aneh,Narasi Newsroom
3,@anggaviali1188,Nonton tanpa baca komen itu lebih baik,Narasi Newsroom
4,@derikoswaranda3500,woyyy mana ini?? ko komenan ga ada yg pro pa w...,Narasi Newsroom
5,@yudihartono7255,"Benarkah ini bagian dr kebebasan berpendapat, ...",Narasi Newsroom
6,@Alzhevi,Jawaban prabowo seperti bercerita dan membantah,Narasi Newsroom
7,@adekoswara6989,"""... apakah demo demo itu murni atau ada yg ba...",Narasi Newsroom
8,@minongbolang,Ditanya apa jawabanya apa😢. Nggak nyambung 😢,Narasi Newsroom
9,@muhammadrizalulloh,11/100 is real ..sangat sabar sekali jurnalis ...,Narasi Newsroom
